# Exercise 24 Solution - Introduction to SINDy

### Task
Implement the sequential thresholded least squares algorithm and perform SINDy on the herein given example and the example from the book

### Learning goals
- Familiarize yourself with SINDy
- Understand the sequential thresholded least squares algorithm 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from scipy.integrate import solve_ivp

## Sequential thresholded least squares algorithm

**solves sparse regression**
$$\dot{\boldsymbol{X}}=\boldsymbol{\Theta}(\boldsymbol{X})\boldsymbol{\Xi}$$

In [ ]:
def sequentialThresholdedLeastSquares(theta, dXdt, k=10, tol=1e-1):
    Xi = np.linalg.lstsq(theta, dXdt, rcond=None)[0]
    for i in range(k):
        smallindices = abs(Xi) < tol
        Xi[smallindices] = 0
        for j in range(len(dXdt[1])):
            bigindices = ~smallindices[:, j]
            Xi[bigindices, j] = np.linalg.lstsq(theta[:, bigindices], dXdt[:, j], rcond=None)[0]
    return Xi

## SINDy class

In [ ]:
class SINDy:
    """A class used for sparse system identification of nonlinear dynamical systems
       Implementation adapted from https://arxiv.org/pdf/1509.03580.pdf"""

    def __init__(self, X, dXdt, theta):
        self.Xi = None
        self.X = X
        self.dXdt = dXdt
        self.theta = theta

    def solveSINDy(self, k=10, tol=1e-1):
        self.Xi = sequentialThresholdedLeastSquares(self.theta(np.transpose(self.X)), self.dXdt, k, tol)
        return self.Xi

    def solveODEs(self, interval, initialValues):
        if self.Xi is not None:
            def rhs(t, y):
                return np.dot(self.theta(y), self.Xi)

            solution = solve_ivp(rhs, interval, initialValues, method='LSODA', rtol=1e-6, min_step=1e-3, max_step=1e-3)
            return solution.t, solution.y
        else:
            return 1, 1

## Pre-processing

**SINDy hyperparameters (for sequential thresholded least squares)**

In [ ]:
tol = 1e-2
k = 10

**helpers for sampling of the snapshot matrix**

In [ ]:
t = np.linspace(0, 2 * np.pi, 10)
t = np.expand_dims(t, axis=1)

In [ ]:
x = np.sin(t) + np.cos(t)
y = np.cos(t)
dxdt = np.cos(t) - np.sin(t)
dydt = -np.sin(t)

# case from the exercise
#x = np.sin(2*t) 
#y = 2*np.cos(2*t) - np.sin(2*t)
#dxdt = 2*np.cos(2*t)
#dydt = -4*np.sin(2*t) - 2*np.cos(2*t)

**collect time history for snapshot matrix**

In [ ]:
X = np.concatenate((x, y), axis=1)
dXdt = np.concatenate((dxdt, dydt), axis=1)

**library of candidate functions**

In [ ]:
theta = lambda X: np.transpose(np.array([X[0] * 0 + 1, X[0], X[1], X[0] ** 2, X[1] ** 2, X[0] * X[1]]))

## SINDy

In [ ]:
model = SINDy(X, dXdt, theta)
Xi = model.solveSINDy(k, tol)
print(Xi)

**prediction**

In [ ]:
tEstimate, XEstimate = model.solveODEs((0, 2 * np.pi), [x[0, 0], y[0, 0]])

## Post-processing

**post-processing helper (plots trajectory based on identified system)**

In [ ]:
def plotEstimate(tSample, XSample, tEstimate, XEstimate):
    color = ['b', 'r', 'gray', 'silver']

    # Set up plot
    fig, ax = plt.subplots()

    # Plot data
    for i in range(len(XSample[0])):
        plt.plot(tSample, XSample[:, i], 'o', color=color[i])
        plt.plot(tEstimate, XEstimate[i], '-', color=color[i], label='$y_{}$'.format(i + 1))

    l1 = plt.Line2D([0], [0], marker='o', lw=0, color='k', label='sample', markersize=12)
    l2 = plt.Line2D([0], [0], color='k', label='prediction')
    handles, labels = plt.gca().get_legend_handles_labels()
    handles.extend([l1, l2])

    ax.set_xlabel("$t$")
    ax.set_ylabel("$y$")
    ax.legend()
    fig.tight_layout()
    plt.show()

**predicted trajectories**

In [ ]:
plotEstimate(t, X, tEstimate, XEstimate)